<a href="https://colab.research.google.com/github/pks24680/DecisionTrees/blob/main/Assignment_2_ML_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
from numpy import log2 as log
from pprint import pprint  #we cannot directly import pprint as it will be calling a class
                            # object within a module object with the same name as the module
import random
#from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files  #I need to import the file as I have locally downloaded it.
                                # I could even do it by uploading it here first and avoid these lines of code but doing this
                                # makes it easier to upload any local file directly here
uploaded=files.upload()

Saving Decision_Tree_ Dataset.csv to Decision_Tree_ Dataset (1).csv


In [ ]:
loan_data=pd.read_csv("Decision_Tree_ Dataset.csv",sep = ',', header= 0)
loan_data #just to check the data once

,1,2,3,4,sum,Unnamed: 5
0,201,10018,250,3046,13515,yes
1,205,10016,395,3044,13660,yes
2,257,10129,109,3251,13746,yes
3,246,10064,324,3137,13771,yes
4,117,10115,496,3094,13822,yes
...,...,...,...,...,...,...
995,413,14914,523,4683,20533,No
996,359,14423,927,4838,20547,No
997,316,14872,613,4760,20561,No
998,305,14926,897,4572,20700,No


In [ ]:
loan_data.rename(columns={'1':'Initial payment', '2':'Last Payment',
                              '3':'Credit Score', '4':'House Number', 'sum': 'Sum',
                            'Unnamed: 5': 'Result'}, inplace= True) #A few data preparation and cleaning steps carried to enhance display
                                                                     # and make data more understandable
loan_data #check if the above worked

,Initial payment,Last Payment,Credit Score,House Number,Sum,Result
0,201,10018,250,3046,13515,yes
1,205,10016,395,3044,13660,yes
2,257,10129,109,3251,13746,yes
3,246,10064,324,3137,13771,yes
4,117,10115,496,3094,13822,yes
...,...,...,...,...,...,...
995,413,14914,523,4683,20533,No
996,359,14423,927,4838,20547,No
997,316,14872,613,4760,20561,No
998,305,14926,897,4572,20700,No


In [ ]:
print("Dataset Shape:: "), loan_data.shape #find the number of rows(instances) and columns in dataset

Dataset Shape:: 


(None, (1000, 6))

In [ ]:
print("Dataset:: ")
loan_data.head() #prints top 5 instances

Dataset:: 


,Initial payment,Last Payment,Credit Score,House Number,Sum,Result
0,201,10018,250,3046,13515,yes
1,205,10016,395,3044,13660,yes
2,257,10129,109,3251,13746,yes
3,246,10064,324,3137,13771,yes
4,117,10115,496,3094,13822,yes


In [ ]:
loan_data.columns #to have a look at the columns we have in our dataset

Index(['Initial payment', 'Last Payment', 'Credit Score', 'House Number',
       'Sum', 'Result'],
      dtype='object')

In [ ]:
del loan_data['Sum'] #Delete unnecessary column to make data more clean as this attribute can be inferred from given info
loan_data

,Initial payment,Last Payment,Credit Score,House Number,Result
0,201,10018,250,3046,yes
1,205,10016,395,3044,yes
2,257,10129,109,3251,yes
3,246,10064,324,3137,yes
4,117,10115,496,3094,yes
...,...,...,...,...,...
995,413,14914,523,4683,No
996,359,14423,927,4838,No
997,316,14872,613,4760,No
998,305,14926,897,4572,No


In [ ]:
X=loan_data.values[:, 1:5] #all data points
Y=loan_data.values[:,4] #only the result attribute values

In [ ]:
def split_for_testing(loan_data, test_size):   #function to split the data- can use pre-defined split functions from libraries
                                       # but this one explains the in-depth process
                                       #takes the dataset and size of test set that we want as parameters

    if isinstance(test_size, float):           #in case we want to represent test size in decimal point/percentage of the dataset
                                               #isinstance checks whether the object is an instanc eof a particular class- returns boolean true/false
        test_size=round(test_size*len(loan_data))

    index=loan_data.index.tolist() #convert to list so it is possible to use later to create the training and testing sets
    test_index= random.sample(population=index,k=test_size) #to randomly select the data for testing

    test_set= loan_data.loc[test_index]  #test dataset
    train_set=loan_data.drop(test_index) #dataset to be used for training

    return train_set, test_set

In [ ]:
random.seed(0) # used in the generation of a pseudo-random encryption key.
train_set, test_set=split_for_testing(loan_data, test_size=300)  #test on 30% of the dataset(can be repeated values)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)- written earlier when using the library to split

In [ ]:
def group_data(loan_data):

    label_column= loan_data[:, -1] #refer to last column- here it is result
    unique_classes, counts_unique_classes= np.unique(label_column, return_counts=True) #find unique values for result- Yes and No

    i= counts_unique_classes.argmax() #Find maximum number of unique values. For result it would just be 2 classes- Yes and No
    result_data= unique_classes[i] #Classify data based on this criteria

    return result_data

In [ ]:
def get_splits(loan_data): #find out the potential values we can split the data through

    _, columns= loan_data.shape #get columns from shape attribute by not providing row value
    possible_splits= {}
    for i in range(columns-1): # excluding result column
        values= loan_data[:, i]
        possible_splits[i]= []
        unique_values=np.unique(values)

        for j in range(len(unique_values)):
            if j!=0: #if there are no unique values, you break out of the inner for loop here
                previous=unique_values[j-1]
                current=unique_values[j]
                split= (current + previous)/2  #split along a line halfway to differentiate 2 data points
                possible_splits[i].append(split)

    return possible_splits


In [ ]:
def split_data(loan_data, split_title, split_value): #this function simply splits data to values lesser than(or equal to) and those greater than
                                                     #the conditional value and then returns these
    split_column= loan_data[:, split_title]
    less_than= loan_data[split_column<=split_value]
    greater_than= loan_data[split_column>split_value]

    return less_than, greater_than


In [ ]:
def is_Pure(loan_data):

    label_column = loan_data[:, -1]
    unique_classes = np.unique(label_column)
    if len(unique_classes) == 1: #if there is only one unique class left, the data is pure
        return True
    else:
        return False


In [ ]:
def entropy_att(loan_data):

    result_column= loan_data[:, -1]
    _, n= np.unique(result_column, return_counts=True)  #count the number of unique values
    probabilities= n/n.sum() # unique values/ total number of values
    entropy= sum(probabilities * -np.log2(probabilities)) # formula for entropy- summation

    return entropy

def entropy(less_than, greater_than ):

    n= len(less_than)+len(greater_than) #total values(denominator)
    p_less_than= len(less_than)/n #probability for less than the split value
    p_greater_than= len(greater_than)/n #proabibility for greater than the split value
    entropy_d=  (p_less_than*entropy_att(less_than) + p_greater_than*entropy_att(greater_than)) # entropy for the dataset- or above the split

    return entropy_d

def best_split(loan_data, possible_splits): #this function selects the best column to split through and the best value for comparison

    entropy_d = 10106651651 #any high value for initial comparison
    for i in possible_splits:
        for j in possible_splits[i]:
            data_below, data_above = split_data(loan_data, split_title=i, split_value=j)
            entropy_now = entropy(data_below, data_above)
            if entropy_now<=entropy_d:
                entropy_d= entropy_now
                best_column= i
                best_value= j

    return best_column, best_value

In [ ]:
def decision_tree(loan_data,counter=0,min_samples=2,max_depth=20):

    if counter==0:
        global columns #we cannot access them if the variables are not global- error encountered and solved using stackoverflow and Youtube
        columns = loan_data.columns
        loan_data = loan_data.values #represent in 2d array format
    else:
        loan_data = loan_data
    if (is_Pure(loan_data))or(len(loan_data)<min_samples)or(counter==max_depth): #any of the stopping conditions
        result_data= group_data(loan_data)

        return result_data
    else:
        counter+= 1
        possible_splits= get_splits(loan_data)   #Use all the functions defined for functionality
        split_column, split_value=best_split(loan_data, possible_splits)
        data_below, data_above= split_data(loan_data, split_column, split_value)
        feature= columns[split_column]
        criteria="{}  <=  {}".format(feature, split_value) #placeholders placed to enter variable values in string- there is double space
                                                                  # so that the splitting of question doesn't split based on single space
                                                                  #otherwise the column titles would be split
        sub_tree={criteria: []} #initial subtree

        yes=decision_tree(data_below, counter, min_samples, max_depth)
        no=decision_tree(data_above, counter, min_samples, max_depth)
        if yes==no:
            sub_tree= yes #We don't need to repeat if the answers to the criteria are the same
        else:
            sub_tree[criteria].append(yes) #keep adding values to this subtree list based on the conditions meeting at each iteration
            sub_tree[criteria].append(no)

        return sub_tree


In [ ]:
tree= decision_tree(train_set, max_depth=6) #modify the max_depth multiple times and check accuracy
                                                      #On doing this we obtain best accuracy(96%) 6 onwards
pprint(tree)

{'Last Payment  <=  12336.0': [{'Last Payment  <=  11343.5': ['yes',
                                                              {'House Number  <=  4351.0': [{'House Number  <=  3936.0': ['yes',
                                                                                                                          {'Credit Score  <=  725.5': ['yes',
                                                                                                                                                       {'Last Payment  <=  11950.5': ['yes',
                                                                                                                                                                                      'No']}]}]},
                                                                                            {'Credit Score  <=  704.5': [{'Last Payment  <=  11892.0': [{'House Number  <=  4991.5': ['yes',
                                                                        

In [ ]:
check = test_set.iloc[0] #test the first random one to check
# iloc[] allows to choose a cell of the dataset-
#it helps pick a value that belongs to a row/column from the data frame
check

Initial payment      474
Last Payment       13961
Credit Score         638
House Number        4090
Result                No
Name: 864, dtype: object

In [ ]:
def group(check,tree): #this function returns the target value- predicted version
    criteria=list(tree.keys())[0]
    feature, comparison_operator, split_value=criteria.split("  ")
    if check[feature]<=float(split_value):
        answer= tree[criteria][0]
    else:
        answer= tree[criteria][1]
    if not isinstance(answer, dict):
        return answer
    else:
        tree_2= answer
        return group(check,tree_2)


In [ ]:
group(check, tree) #checking just one of the test rows

'No'

In [ ]:
def accuracy_s(loan_data, tree): #to get how accurate our algorithm/data model is

    loan_data["classification"]=loan_data.apply(group, axis=1, args=(tree,))
    loan_data["classification_c"]=loan_data["classification"]==loan_data["Result"] #if the prediction equals the result in the known set,
                                                                                             #then the classification was done correctly
    accuracy=loan_data["classification_c"].mean() #take the mean/average value

    return accuracy

In [ ]:
accuracy=accuracy_s(test_set, tree) #provide it now with the test set as input to see how accurate it is
accuracy

0.96